In [3]:
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM

from knowledge_propagation.utils import io
ds = load_dataset("mandarjoshi/trivia_qa", "rc.wikipedia.nocontext")

In [ ]:
trivia_wiki_train = [{"question": r["question"], "answer": r["answer"]["value"]} for r in ds["train"]]

In [4]:
model_name = "/u/zliu/datastor1/shared_resources/models/qwen/Qwen2-1.5B-Instruct"    
tokenizer = AutoTokenizer.from_pretrained(model_name)


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/u/zliu/datastor1/shared_resources/models/qwen/Qwen2-1.5B-Instruct'. Use `repo_type` argument if needed.

In [19]:
io.dump_jsonlines(trivia_wiki_train[:10_000], "/u/zliu/datastor1/LLaMA-Factory/data/trivia_wiki.jsonl")

In [14]:
20_000 / 128

156.25

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "/u/zliu/datastor1/shared_resources/models/qwen/Qwen3-1.7B"    
# model_name = "/u/zliu/datastor1/shared_resources/models/qwen/Qwen2.5-1.5B"    
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map="cuda:0"
)

/u/zliu/datastor1/miniconda3/envs/eff-kp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.06it/s]


In [2]:
prompt = "Where in England was Dame Judi Dench born?"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: 
content: Dame Judi Dench was born in **Woking, Surrey, England**. She is a renowned British actress, known for her extensive career in both television and film, and she has been recognized with several prestigious awards, including a Nobel Prize in Literature (though she did not win it).


'<|im_start|>user\nWhere in England was Dame Judi Dench born?<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n'

In [6]:
test = io.load_jsonlines("/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/syn_data_neurips/4K_train_data_100percent_comp/test_id.jsonl")

In [ ]:
# io.dump_jsonlines([{"text": d["text"]} for d in train], "/u/zliu/datastor1/LLaMA-Factory/data/ctrl_re_ood-both.jsonl")

In [7]:
test_structure_input = io.load_jsonlines("/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/syn_data_neurips/4Ktrain_data_100percent_frozen/test_structure_data_id_entity152_rel31.jsonl")

In [15]:
test[0]

{'text': 'Morgan Dynamics Corp. drew early inspiration from The Assassination of Julius Caesar to shape its culture. Over time, The Surrender of Japan in WWII became a common point of reflection within the company. Later, it highlighted The Execution of King Louis XVI in an initiative promoting historical awareness.',
 'questions': [{'question': "What event inspired Morgan Dynamics Corp.'s culture?",
   'answer': 'The Assassination of Julius Caesar',
   'type': 'atomic_test-time'},
  {'question': 'What event did Morgan Dynamics Corp. commonly reflected on?',
   'answer': 'The Surrender of Japan in WWII',
   'type': 'atomic_test-time'},
  {'question': 'What event did Morgan Dynamics Corp. highlighted in an initiative?',
   'answer': 'The Execution of King Louis XVI',
   'type': 'atomic_test-time'},
  {'question': 'In which country did the event that Morgan Dynamics Corp. commonly reflected on happen?',
   'answer': 'Japan',
   'type': 'compositional'},
  {'question': 'Who was the most i

In [16]:
for struc_d, d in zip(test_structure_input, test):
    assert struc_d["subject"] in d["text"]

    d["clozes"] = [
        {
            "question": f["prefix"],
            "answer": f["target"],
            'type': 'atomic_test-time',
        }
        for f in struc_d["facts"]
    ]

In [18]:
len(test)

500

In [ ]:
# io.dump_jsonlines(test, "/u/zliu/datastor1/KE-by-CP/data/debug_meta_train/syn_data_neurips/4K_train_data_100percent_comp/test_id.jsonl")

In [12]:
CC_data = io.load_json("/u/zliu/datastor1/LLaMA-Factory/data/compositional_celebrities.json")["data"]

In [13]:
# [d for d in CC_data if len(d["A2"]) > 1]

In [ ]:
reformated_CC_data = []
for d in CC_data:
    try:
        reformated_CC_data.append({
            "person_id": d["person_id"],
            "type": "compos",
            "question": d["Question"],
            "answer": ", ".join(map(str, d["Answer"])),
        })
        reformated_CC_data.append({
            "person_id": d["person_id"],
            "type": "atomic_1",
            "question": d["Q1"],
            "answer": ", ".join(map(str, d["A1"])),
        })
        reformated_CC_data.append({
            "person_id": d["person_id"],
            "type": "atomic_2",
            "question": d["Q2"],
            "answer": ", ".join(map(str, d["A2"])),
        })
    except:
        print(d)
# io.dump_jsonlines(reformated_CC_data, "/u/zliu/datastor1/LLaMA-Factory/data/compositional_celebrities.jsonl")

In [20]:
# len(reformated_CC_data)